In [1]:
#from os.path import join as pjoin
#import numpy as np
#import nibabel as nb # plotting remove later
#import matplotlib.pyplot as plt # plotting remove later
#from nilearn import image, plotting # plotting, removable?
#from nilearn.masking import intersect_masks
from nipype import config
from nipype.interfaces.fsl.maths import TemporalFilter
from nipype.interfaces.fsl.model import MELODIC
from nipype.interfaces.fsl.preprocess import SUSAN
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.pipeline.engine import Workflow, Node # MapNode, 
#from tqdm.notebook import tqdm

220422-17:33:39,842 nipype.utils WARNING:
	 A newer version (1.7.1) of nipy/nipype is available. You are using 1.7.0


In [2]:
config.enable_debug_mode()

In [3]:
def calc_susan_thresh(boldfile, maskfile, timeax=0, median_factor=.75):
    """
    Calculate the median value within brainmask and multiply with fixed factor to get an estimate of the contrast
    between background and brain for FSL's SUSAN.
    """
    from nilearn.masking import apply_mask
    import numpy as np
    data = apply_mask(boldfile, maskfile)
    med = np.median(data.mean(axis=timeax))
    del data  # suspect memory leak
    return med * median_factor

In [ ]:
# TODO: Do preprocessing and ICA

# Required inputs:
# - functional run (bold file)
# - brain mask

# Desired outputs:
# - A nipype workflow that combines precessing (smoothing/temporal filtering) 
# and ICA for one functional run

In [8]:
def make_subject_ica_wf():
    """
    TODO: give TR as input (to this function) or infer from data?
    
    Example Inputs:
        wf.inputs.inputspec.bold_file = '/../preproc_bold.nii.gz'
        wf.inputs.inputspec.mask_file = '/../brain_mask.nii.gz'
        wf.inputs.inputspec.hpf_sec = 120.0
        wf.inputs.inputspec.tr = 1.5
        wf.inputs.inputspec.fwhm = 4.0
        wf.inputs.inputspec.out_dir = '/results/melodic'

        wf.base_dir = '/..'
    
    Output:
        nipype workflow = combining precessing (smoothing/temporal filtering)
        and ICA for one functional run
    """
    # create input spec
    inputspec = Node(
        IdentityInterface(
            fields=['bold_file',
                    'mask_file',
                    'hpf_sec',
                    'tr',
                    'fwhm',
                    'out_dir']
                    ),
            name="inputspec")
    
    # create node for smoothing
    calcthresh = Node(
        Function(
            function=calc_susan_thresh, input_names=['boldfile', 'maskfile'], 
            output_names=['smooth_thresh']),
            name='calcthresh'
    )
    susan = Node(SUSAN(fwhm=fwhm), name='susan') # requires 
    # ... temporal filtering
    tfilt = Node(TemporalFilter(highpass_sigma=float(hpf_sec/tr)), name='tfilt')
    # ... ICA
    melodic = Node(MELODIC(tr_sec=tr, out_all=True, no_bet=True, report=True), name='melodic')
    
    # connect nodes in workflow
    wf = Workflow(name='melodicwf')
    wf.connect([
        (inputspec, calcthresh, [('bold_file', 'boldfile'),
                                 ('mask_file', 'maskfile')]),
        (inputspec, susan, [('bold_file', 'in_file')]),
        (inputspec, melodic, [('mask_file', 'mask'),
                              ('out_dir', 'out_dir')]), 
        (calcthresh, susan, [('smooth_thresh', 'brightness_threshold')]),
        (susan, tfilt, [('smoothed_file', 'in_file')]),
        (tfilt, melodic, [('out_file', 'in_files')])
    ])
    return wf

# Tasks

- Add inputspec and outputspec to our workflow!
- execute for a bunch of runs/subjects/datasets!
- Put the code we wrote today in a python file in our repo for future happy uses!!!

### Paths and workflow execution

In [5]:
# hardcoded paths
bold_file = '/LOCAL/jzerbe/faces_vs_houses/derivatives/fmriprep/sub-08/func/sub-08_task-effort_space-T1w_desc-preproc_bold.nii.gz'
mask_file = '/LOCAL/jzerbe/faces_vs_houses/derivatives/fmriprep/sub-08/func/sub-08_task-effort_space-T1w_desc-brain_mask.nii.gz'
out_dir = '/LOCAL/jzerbe/temp_results/melodic/sub-08_task-effort_space-T1w_melodic'

# parameter
hpf_sec =120.0
tr = 1.5
fwhm = 4.0
base_dir = '/LOCAL/jzerbe/temp_results'

In [9]:
melodicwf = make_subject_ica_wf()

# input to workflow
melodicwf.inputs.inputspec.bold_file = bold_file
melodicwf.inputs.inputspec.mask_file = mask_file
melodicwf.inputs.inputspec.hpf_sec = hpf_sec
melodicwf.inputs.inputspec.tr = tr
melodicwf.inputs.inputspec.fwhm = fwhm
melodicwf.inputs.inputspec.out_dir = out_dir

melodicwf.base_dir = base_dir

220422-17:34:55,513 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.calcthresh): No edge data
220422-17:34:55,514 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.calcthresh): new edge data: {'connect': [('bold_file', 'boldfile'), ('mask_file', 'maskfile')]}
220422-17:34:55,515 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.susan): No edge data
220422-17:34:55,515 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.susan): new edge data: {'connect': [('bold_file', 'in_file')]}
220422-17:34:55,516 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.melodic): No edge data
220422-17:34:55,516 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.melodic): new edge data: {'connect': [('mask_file', 'mask'), ('out_dir', 'out_dir')]}
220422-17:34:55,516 nipype.workflow DEBUG:
	 (melodicwf.calcthresh, melodicwf.susan): No edge data
220422-17:34:55,517 nipype.workflow DEBUG:
	 (melodicwf.calcthresh, melodicwf.susan): new edge data: {'connect': [('smooth_t

In [10]:
melodicwf.run()

220422-17:34:59,491 nipype.workflow DEBUG:
	 Creating flat graph for workflow: melodicwf
220422-17:34:59,494 nipype.workflow DEBUG:
	 expanding workflow: melodicwf
220422-17:34:59,496 nipype.workflow DEBUG:
	 processing node: melodicwf.inputspec
220422-17:34:59,497 nipype.workflow DEBUG:
	 processing node: melodicwf.calcthresh
220422-17:34:59,497 nipype.workflow DEBUG:
	 processing node: melodicwf.susan
220422-17:34:59,498 nipype.workflow DEBUG:
	 processing node: melodicwf.melodic
220422-17:34:59,498 nipype.workflow DEBUG:
	 processing node: melodicwf.tfilt
220422-17:34:59,499 nipype.workflow DEBUG:
	 finished expanding workflow: melodicwf
220422-17:34:59,500 nipype.workflow INFO:
	 Workflow melodicwf settings: ['check', 'execution', 'logging', 'monitoring']
220422-17:34:59,502 nipype.workflow DEBUG:
	 PE: expanding iterables
220422-17:34:59,502 nipype.workflow DEBUG:
	 [Node] calcthresh - setting input boldfile = /LOCAL/jzerbe/faces_vs_houses/derivatives/fmriprep/sub-08/func/sub-08_t

220422-17:34:59,551 nipype.workflow DEBUG:
	 output: out_file
220422-17:34:59,551 nipype.workflow DEBUG:
	 [Node] melodic - setting input in_files = /LOCAL/jzerbe/temp_results/melodicwf/tfilt/sub-08_task-effort_space-T1w_desc-preproc_bold_smooth_filt.nii.gz
220422-17:34:59,553 nipype.workflow DEBUG:
	 [Node] Hashes: [('in_files', [('/LOCAL/jzerbe/temp_results/melodicwf/tfilt/sub-08_task-effort_space-T1w_desc-preproc_bold_smooth_filt.nii.gz', 'fc35d53c72bb0ccb929af3980233959c')]), ('mask', ('/LOCAL/jzerbe/faces_vs_houses/derivatives/fmriprep/sub-08/func/sub-08_task-effort_space-T1w_desc-brain_mask.nii.gz', 'a83ffdb9e86682275b9907e54f66bd7f')), ('no_bet', True), ('out_all', True), ('out_dir', '/LOCAL/jzerbe/temp_results/melodic/sub-08_task-effort_space-T1w_melodic'), ('output_type', 'NIFTI_GZ'), ('report', True), ('tr_sec', '1.5000000000')], a7c8ae8e9b01e2a814be1edb84cd74b1, /LOCAL/jzerbe/temp_results/melodicwf/melodic/_0xa7c8ae8e9b01e2a814be1edb84cd74b1.json, ['/LOCAL/jzerbe/temp_result

### Plotting

In [ ]:
melodicimg = '/LOCAL/jzerbe/temp_results/melodic/sub-08_task-effort_space-T1w_melodic/melodic_IC.nii.gz'
#view_img(img)
anatimg = '/LOCAL/jzerbe/faces_vs_houses/derivatives/fmriprep/sub-08/anat/sub-08_desc-preproc_T1w.nii.gz'
def plot_slice(fname):

    # Load the image
    img = nb.load(fname)
    data = img.get_data()

    # Cut in the middle of the brain
    cut = int(data.shape[-1]/2) + 10

    # Plot the data
    plt.imshow(np.rot90(data[..., cut]), cmap="gray")
    plt.gca().set_axis_off()

#plotting.plot_epi(
#    anatimg,
#    title="T1", display_mode='ortho', annotate=False, draw_cross=False, cmap='gray');

print(image.load_img(melodicimg).shape)
first_melodic = image.index_img(melodicimg, 0)
plt.figure(figsize=(20,40))
plotting.plot_stat_map(first_melodic, bg_img=anatimg)


In [ ]:
# long plot with brain slices from bottom to top


# frequency of the IC



In [ ]:
nilearn.plotting.displays._slicers.OrthoSlicer?

In [ ]:
selected_volumes = image.index_img(melodicimg, slice(3, 5))

for img in image.iter_img(selected_volumes):
    # img is now an in-memory 3D img
    plotting.plot_stat_map(img, threshold=3, display_mode="z", cut_coords=1,
                           colorbar=False, bg_img=anatimg)